# Part 3: Unbiased Evaluation using a New Test Set

In this part, we are given a new test set (`/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`). We can now take advantage of the entire smart sample that we created in Part I. 

* Retrain a pipeline using the optimal parameters that the pipeline learned. We don't need to repeat GridSearch here. 

## Import modules as needed

In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import os, sys
import itertools
import numpy as np
import pandas as pd

from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

## Load smart sample and the best pipeline from Part II

In [2]:
import joblib
model3 = joblib.load('bestmodel.pkl')
best_pipeline3 = model3.best_estimator_

In [3]:
df = joblib.load('dataset_new.pkl')
df.head()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
197,0.0,2.0,0.0,54.0,54.0,54.0,0.0,0.0,0.0,0.0,...,0.0,1.00,0.98,0.0,1,0,1,1,0,1
606,0.0,2.0,0.0,2.0,2.0,2.0,3.0,3.0,3.0,4.0,...,0.0,0.73,0.77,0.0,0,0,0,1,0,1
846,1.0,12.0,0.0,18.0,18.0,18.0,1.0,2.0,8.0,15.0,...,0.0,0.82,0.79,0.0,0,0,0,1,0,1
882,-2.0,8.0,0.0,17.0,23.0,33.0,2.0,12.0,19.0,25.0,...,0.0,0.37,0.34,2.0,0,0,0,1,0,1
898,0.0,8.0,0.0,30.0,45.0,60.0,11.0,26.0,42.0,73.0,...,0.0,0.82,0.81,0.0,0,0,0,1,0,1


In [4]:
print(df.columns)

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')



##  Retrain a pipeline using the full sampled training data set

Use the full sampled training data set to train the pipeline.

In [5]:
# Add code below this comment  (Question #E301)
# ----------------------------------


X = df[[ 'national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop']]
y = df[['went_on_backorder']]




In [6]:
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

In [7]:
# Fit the model and make predictions
y_pred_out = lof.fit_predict(X)

In [8]:
# Get the number of outliers detected
n_outliers = np.sum(y_pred_out == -1)


In [9]:
# Print the number of outliers detected
print("Number of outliers detected: ", n_outliers)

Number of outliers detected:  2256


# retraining

In [10]:
best_pipeline3.fit(X,y)

/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_s

Pipeline(steps=[('scale', MinMaxScaler()),
                ('rfe',
                 RFE(estimator=RandomForestClassifier(),
                     n_features_to_select=15)),
                ('rf', RandomForestClassifier(max_depth=20, n_estimators=200))])

### Save the trained model with the pickle library.

In [11]:
# Add code below this comment  
# -----------------------------

joblib.dump( best_pipeline3.fit , 'retrain_model_3.pkl')




['retrain_model_3.pkl']


## Load the Testing Data and evaluate your model

 * `/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`
 
* We need to preprocess this test data (**follow** the steps similar to Part I)
* **If you have fitted any normalizer/standardizer in Part 2, then we have to transform this test data using the fitted normalizer/standardizer!**

In [12]:
# Preprocess the given test set  (Question #E302)
# ----------------------------------

#read the dataset
test_model = pd.read_csv("/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv")


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### preprocessing of data

In [13]:
#yes/no column
yes_no_columns = list(filter(lambda i: test_model[i].dtype!=np.float64, test_model.columns))
print(yes_no_columns)

for i in (yes_no_columns):
    a = test_model[i].unique()
    print("column  - {} has {}".format(i, a))
    print(test_model['sku'].dtype)
test_model['sku'] = pd.to_numeric(test_model['sku'], errors='coerce')
print(test_model['sku'].dtype)
print(test_model.shape)



['sku', 'potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']
column  - sku has [3285085 3285131 3285358 ... '3526990' '3526991' '(242075 rows)']
object
column  - potential_issue has ['No' 'Yes' nan]
object
column  - deck_risk has ['Yes' 'No' nan]
object
column  - oe_constraint has ['No' 'Yes' nan]
object
column  - ppap_risk has ['No' 'Yes' nan]
object
column  - stop_auto_buy has ['Yes' 'No' nan]
object
column  - rev_stop has ['No' 'Yes' nan]
object
column  - went_on_backorder has ['No' 'Yes' nan]
object
float64
(242076, 23)


In [14]:
#filling missing value
    
    
yes_no_columns = list(filter(lambda i: test_model[i].dtype!=np.float64, test_model.columns))
print(yes_no_columns)
for column_name in yes_no_columns:
    mode = test_model[column_name].apply(str).mode()[0]
    print('Filling missing values of {} with {}'.format(column_name, mode))
    test_model[column_name].fillna(mode, inplace=True)
for i in yes_no_columns:
    test_model[i].replace(('Yes', 'No'), (1, 0), inplace=True)
    print(i, test_model[i].unique())

['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']
Filling missing values of potential_issue with No
Filling missing values of deck_risk with No
Filling missing values of oe_constraint with No
Filling missing values of ppap_risk with No
Filling missing values of stop_auto_buy with Yes
Filling missing values of rev_stop with No
Filling missing values of went_on_backorder with No
potential_issue [0 1]
deck_risk [1 0]
oe_constraint [0 1]
ppap_risk [0 1]
stop_auto_buy [1 0]
rev_stop [0 1]
went_on_backorder [0 1]


In [15]:
test_model= test_model.dropna()

In [16]:
print(test_model.went_on_backorder.sum())

2604


We can now predict and evaluate with the preprocessed test set. It would be interesting to see the performance with and without outliers removal from the test set. We can report confusion matrix, precision, recall, f1-score, accuracy, and other measures (if any). 

In [17]:
# Add code below this comment  (Question #E303)
# ----------------------------------




X_test= test_model[['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop']]
y_test = test_model[["went_on_backorder"]]







In [18]:
y_test = test_model[["went_on_backorder"]]

In [19]:
y_pred = model3.predict(X_test)

In [20]:
#confusion matrix

from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test, y_pred)

In [21]:
print(confusion_matrix)

[[197304  27443]
 [   495   2109]]


In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.88      0.93    224747
           1       0.07      0.81      0.13      2604

    accuracy                           0.88    227351
   macro avg       0.53      0.84      0.53    227351
weighted avg       0.99      0.88      0.92    227351



## Reflect

Imagine you are data scientist that has been tasked with developing a system to save your 
company money by predicting and preventing back orders of parts in the supply chain.

Write a **brief summary** for "management" that details your findings, 
your level of certainty and trust in the models, 
and recommendations for operationalizing these models for the business.

# Save your notebook!
## Then `File > Close and Halt`